# 유의어 랜덤 추가

In [ ]:
!pip install konlpy

In [33]:
import gensim
from konlpy.tag import Okt
import pandas as pd
import random

In [211]:
data = pd.read_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/train_normalization.csv')
data.head()

,class,label_sentence,non_label_sentence,binary_class
0,직장 내 괴롭힘 대화,1:길동 씨 이번에 이것 좀 처리해요\n2:이거 제가 한 게 아닌데요\n1:팀에서 ...,길동 씨 이번에 이것 좀 처리해요\n이거 제가 한 게 아닌데요\n팀에서 내가 네가 ...,공격 대화
1,기타 괴롭힘 대화,1:야 너 앞니 왜 그렇게 튀어나왔냐\n2:태어날 때부터 그랬어 물어보지 마\n1:...,야 너 앞니 왜 그렇게 튀어나왔냐\n태어날 때부터 그랬어 물어보지 마\n아 그럼 태...,공격 대화
2,갈취 대화,1:원후야 학원 교재 사야 되는데 8만 원만 주면 안 돼\n2:미안 나 오늘은 진짜...,원후야 학원 교재 사야 되는데 8만 원만 주면 안 돼\n미안 나 오늘은 진짜 돈이 ...,공격 대화
3,갈취 대화,1:너 저번에 술 먹은 날 기억해\n2:아니 왜\n1:야 300만 원만 가져와\n2...,너 저번에 술 먹은 날 기억해\n아니 왜\n야 300만 원만 가져와\n뭔 소리야 ...,공격 대화
4,협박 대화,1:너 또 내말 무시하냐\n1:이 새끼 널 좆으로 보나 본데\n2:아냐 진짜 시간이...,너 또 내말 무시하냐\n이 새끼 널 좆으로 보나 본데\n아냐 진짜 시간이 없었어 미...,공격 대화


In [17]:
data.shape

(3870, 4)

In [4]:
model = gensim.models.Word2Vec.load('/content/drive/MyDrive/아이펠톤/ko.bin')

In [22]:
model.most_similar("돈")[0][0]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


'현상금'

In [71]:
def random_insertion(sen):
    noun = []
    origin = okt.morphs(sen)
    tokens = okt.pos(sen, stem=True, norm=True)
    for word in tokens:
        if word[1] in ['Noun']:
            noun.append(word[0])

    simi = []
    for n in noun:
        try:
            simi.append(model.most_similar(n)[0][0])
        except:
            continue

    ran_idx = random.randint(0,len(origin))

    for s in simi:
        origin.insert(ran_idx, s)

    return ' '.join(origin)

In [85]:
def augmen_conversation(sen):
    new_sen = []
    sen_split = sen.split('\n')
    for sen in sen_split:
        new_sen.append(random_insertion(sen)+'\n')

    new_sen = ' '.join(new_sen)
    new_sen = new_sen[:-1]
    return new_sen

In [94]:
sen = '길동 씨 이번에 이것 좀 처리해요'
random_insertion(sen)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  if sys.path[0] == '':


'길동 씨 이번 에 이 것 좀 처리 입력 조금 방법 저번 정씨 신길 해요'

In [100]:
origin = list(data.non_label_sentence)

insert_conversation = []

for ori in origin:
    insert_conversation.append(augmen_conversation(ori))

insert_data = pd.DataFrame(zip(insert_conversation))
insert_data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  if sys.path[0] == '':


,0
0,길동 입력 조금 방법 저번 정씨 신길 씨 이번 에 이 것 좀 처리 해요\n 이 거 ...
1,야 너 과연 에드 앞니 왜 그렇게 튀어나왔냐\n 마가 때쯤 태어날 때 부터 그랬어 ...
2,원후 야 학원 교재 사야 되는데 못가 돈독 교과서 학교 8만 원만 주면 안 돼\n ...
3,회상 첫날 독약 지난번 에드 너 저번 에 술 먹은 날 기억 해\n 아니 왜 과연\n...
4,너 또 배제 말로 각기 에드 내 말 무시 하냐\n 이 새끼 널 좆 널이 암컷 그러 ...
...,...
3865,준 하야 넌 대가리 가 왜 과연 플리트 스즈키 우승자 이렇게 크냐\n 내 머리 가 ...
3866,내 가 지금 너 아들 김길준 데리 고 있어 살리고 싶으면 계좌 에 1억만 예금 거느...
3867,나 는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살 지 살아오 젊은이 정씨 ...
3868,누구 맘대로 여기 서 장사 태수 알리 어디 하래\n 이 게 입히 어찌 그러 무슨 일...


In [121]:
cla = pd.DataFrame(data['class'])
data_insertion = pd.concat([insert_data,cla],axis=1)
data_insertion.rename(columns = {0:'non_label_sentence'}, inplace=True)
data_insertion.head()

,non_label_sentence,class
0,길동 입력 조금 방법 저번 정씨 신길 씨 이번 에 이 것 좀 처리 해요\n 이 거 ...,직장 내 괴롭힘 대화
1,야 너 과연 에드 앞니 왜 그렇게 튀어나왔냐\n 마가 때쯤 태어날 때 부터 그랬어 ...,기타 괴롭힘 대화
2,원후 야 학원 교재 사야 되는데 못가 돈독 교과서 학교 8만 원만 주면 안 돼\n ...,갈취 대화
3,회상 첫날 독약 지난번 에드 너 저번 에 술 먹은 날 기억 해\n 아니 왜 과연\n...,갈취 대화
4,너 또 배제 말로 각기 에드 내 말 무시 하냐\n 이 새끼 널 좆 널이 암컷 그러 ...,협박 대화


In [122]:
ori = data[['non_label_sentence', 'class']]

In [ ]:
last = pd.concat([ori, data_insertion]).reset_index(drop=True)
last

In [130]:
last.to_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/train_insertion.csv', index=False)

# 원래단어 바로뒤 유의어 추가

In [160]:
def random_insertion(sen):
    noun = []
    origin = okt.morphs(sen)
    tokens = okt.pos(sen, stem=True, norm=True)
    for word in tokens:
        if word[1] in ['Noun']:
            noun.append(word[0])

    for n in noun:
        try:
            s = model.most_similar(n)[0][0]
            i = origin.index(n)
            origin.insert(i+1, s)
        except:
            continue

    return ' '.join(origin)

In [161]:
def augmen_conversation(sen):
    new_sen = []
    sen_split = sen.split('\n')
    for sen in sen_split:
        new_sen.append(random_insertion(sen)+'\n')

    new_sen = ' '.join(new_sen)
    new_sen = new_sen[:-1]
    return new_sen

In [164]:
origin = list(data.non_label_sentence)

insert_conversation = []

for ori in origin:
    insert_conversation.append(augmen_conversation(ori))

insert_data = pd.DataFrame(zip(insert_conversation))
insert_data.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  # This is added back by InteractiveShellApp.init_path()


,0
0,길동 신길 씨 정씨 이번 저번 에 이 것 방법 좀 조금 처리 입력 해요\n 이 거 ...
1,야 너 에드 앞니 왜 과연 그렇게 튀어나왔냐\n 태어날 때 때쯤 부터 그랬어 물어보...
2,원후 야 학원 학교 교재 교과서 사야 되는데 8만 원만 돈독 주면 안 못가 돼\n ...
3,너 에드 저번 지난번 에 술 독약 먹은 날 첫날 기억 회상 해\n 아니 왜 과연\n...
4,너 에드 또 각기 내 말 말로 무시 배제 하냐\n 이 그러 새끼 암컷 널 널이 좆 ...


In [166]:
insert_data.rename(columns = {0:'non_label_sentence'}, inplace=True)
add_data = data.copy()
add_data = add_data.drop('non_label_sentence', axis=1)
new = pd.concat([add_data,insert_data], axis=1).reset_index(drop=True)
new = new[['class', 'label_sentence', 'non_label_sentence', 'binary_class']]
new_100 = pd.concat([data,new]).reset_index(drop=True)

In [174]:
new_100.to_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/train_contextualized_RI_100.csv', index=False)
pd.read_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/train_contextualized_RI_100.csv')

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/train_contextualized_RI_100.csv')
data[3870

In [214]:
idx = list(range(3870))

In [215]:
data = data.drop(idx) 

In [218]:
new = data.reset_index(drop=True)

In [220]:
new.to_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/train_contextualized_RI.csv', index=False)
pd.read_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/train_contextualized_RI.csv')

,class,label_sentence,non_label_sentence,binary_class
0,직장 내 괴롭힘 대화,1:길동 씨 이번에 이것 좀 처리해요\n2:이거 제가 한 게 아닌데요\n1:팀에서 ...,길동 신길 씨 정씨 이번 저번 에 이 것 방법 좀 조금 처리 입력 해요\n 이 거 ...,공격 대화
1,기타 괴롭힘 대화,1:야 너 앞니 왜 그렇게 튀어나왔냐\n2:태어날 때부터 그랬어 물어보지 마\n1:...,야 너 에드 앞니 왜 과연 그렇게 튀어나왔냐\n 태어날 때 때쯤 부터 그랬어 물어보...,공격 대화
2,갈취 대화,1:원후야 학원 교재 사야 되는데 8만 원만 주면 안 돼\n2:미안 나 오늘은 진짜...,원후 야 학원 학교 교재 교과서 사야 되는데 8만 원만 돈독 주면 안 못가 돼\n ...,공격 대화
3,갈취 대화,1:너 저번에 술 먹은 날 기억해\n2:아니 왜\n1:야 300만 원만 가져와\n2...,너 에드 저번 지난번 에 술 독약 먹은 날 첫날 기억 회상 해\n 아니 왜 과연\n...,공격 대화
4,협박 대화,1:너 또 내말 무시하냐\n1:이 새끼 널 좆으로 보나 본데\n2:아냐 진짜 시간이...,너 에드 또 각기 내 말 말로 무시 배제 하냐\n 이 그러 새끼 암컷 널 널이 좆 ...,공격 대화
...,...,...,...,...
3865,기타 괴롭힘 대화,1:준하야 넌 대가리가 왜 이렇게 크냐\n2:내 머리가 뭐\n1:밥 먹으면 대가리만...,준 우승자 하야 스즈키 넌 플리트 대가리 가 왜 과연 이렇게 크냐\n 내 내고 머리...,공격 대화
3866,갈취 대화,1:내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n2...,내 내고 가 지금 현재 너 에드 아들 딸 김길준 데리 거느리 고 있어 살리고 싶으면...,공격 대화
3867,직장 내 괴롭힘 대화,1:나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n1:왜 그래 들...,나 나의 는 씨 정씨 같은 사람 젊은이 보면 참 신기하더라 어떻게 저렇게 살 살아오...,공격 대화
3868,갈취 대화,1:누구 맘대로 여기서 장사하래\n2:이게 무슨 일입니까\n1:남의 구역에서 장사하...,누구 어디 맘대로 여기 알리 서 장사 태수 하래\n 이 그러 게 무슨 어찌 일 입 ...,공격 대화


# 노이즈추가10%

In [201]:
percent = 0.1
n = list(range(data.shape[0]))
ran_idx = random.sample(n, int(data.shape[0]*percent))
ran_idx = sorted(ran_idx)
len(ran_idx)

387

In [203]:
choied_data = data.loc[ran_idx]

In [205]:
origin = list(choied_data.non_label_sentence)

insert_conversation = []

for ori in origin:
    insert_conversation.append(augmen_conversation(ori))

insert_data = pd.DataFrame(zip(insert_conversation))
insert_data.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  # This is added back by InteractiveShellApp.init_path()


,0
0,안녕하십니까 사장 부사장 님\n 뭔 안녕해 미친 놈 녀석 아 저리 졀 꺼져\n 죄송...
1,네 카나 길동 신길 샐러드 디저트\n 아 저기 불쌍히 요 제 저 가 아까 거기 여기...
2,아가씨 엄마 택시 운전사 비 빌 가 없는데 돈 현상금 좀 조금 빌려\n 얼마 며칠 ...
3,어이 손색 거기 여기저기\n 저 제 저희 그분 조 조의\n 그래 거기 여기저기 너 ...
4,너 에드 왜 과연 내 내고 욕 욕계 하고 다니냐\n 무슨 어찌 소리 울음소리 야\n...


In [209]:
insert_data.rename(columns = {0:'non_label_sentence'}, inplace=True)
add_data = choied_data.copy()
add_data = add_data.drop('non_label_sentence', axis=1)
new = pd.concat([add_data,insert_data], axis=1).reset_index(drop=True)
new = new[['class', 'label_sentence', 'non_label_sentence', 'binary_class']]
new_10 = pd.concat([data,new]).reset_index(drop=True)
new_10.head()

,class,label_sentence,non_label_sentence,binary_class
0,직장 내 괴롭힘 대화,1:길동 씨 이번에 이것 좀 처리해요\n2:이거 제가 한 게 아닌데요\n1:팀에서 ...,길동 씨 이번에 이것 좀 처리해요\n이거 제가 한 게 아닌데요\n팀에서 내가 네가 ...,공격 대화
1,기타 괴롭힘 대화,1:야 너 앞니 왜 그렇게 튀어나왔냐\n2:태어날 때부터 그랬어 물어보지 마\n1:...,야 너 앞니 왜 그렇게 튀어나왔냐\n태어날 때부터 그랬어 물어보지 마\n아 그럼 태...,공격 대화
2,갈취 대화,1:원후야 학원 교재 사야 되는데 8만 원만 주면 안 돼\n2:미안 나 오늘은 진짜...,원후야 학원 교재 사야 되는데 8만 원만 주면 안 돼\n미안 나 오늘은 진짜 돈이 ...,공격 대화
3,갈취 대화,1:너 저번에 술 먹은 날 기억해\n2:아니 왜\n1:야 300만 원만 가져와\n2...,너 저번에 술 먹은 날 기억해\n아니 왜\n야 300만 원만 가져와\n뭔 소리야 ...,공격 대화
4,협박 대화,1:너 또 내말 무시하냐\n1:이 새끼 널 좆으로 보나 본데\n2:아냐 진짜 시간이...,너 또 내말 무시하냐\n이 새끼 널 좆으로 보나 본데\n아냐 진짜 시간이 없었어 미...,공격 대화


In [210]:
new_10.shape

(4609, 4)